In [20]:
!pip  -q install datasets sentence-transformers faiss-cpu langchain



In [2]:
!pip install datasets sentence-transformers faiss-cpu transformers


In [5]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch


In [4]:
dataset = load_dataset("hotpotqa/hotpot_qa", "distractor") # or "fullwiki"

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


hotpot_qa.py:   0%|          | 0.00/6.42k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.19k [00:00<?, ?B/s]

The repository for hotpotqa/hotpot_qa contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/hotpotqa/hotpot_qa.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/90447 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/7405 [00:00<?, ? examples/s]

In [6]:
print("Sample data: ", dataset['train'][0])

Sample data:  {'id': '5a7a06935542990198eaf050', 'question': "Which magazine was started first Arthur's Magazine or First for Women?", 'answer': "Arthur's Magazine", 'type': 'comparison', 'level': 'medium', 'supporting_facts': {'title': ["Arthur's Magazine", 'First for Women'], 'sent_id': [0, 0]}, 'context': {'title': ['Radio City (Indian radio station)', 'History of Albanian football', 'Echosmith', "Women's colleges in the Southern United States", 'First Arthur County Courthouse and Jail', "Arthur's Magazine", '2014–15 Ukrainian Hockey Championship', 'First for Women', 'Freeway Complex Fire', 'William Rast'], 'sentences': [["Radio City is India's first private FM radio station and was started on 3 July 2001.", ' It broadcasts on 91.1 (earlier 91.0 in most cities) megahertz from Mumbai (where it was started in 2004), Bengaluru (started first in 2001), Lucknow and New Delhi (since 2003).', ' It plays Hindi, English and regional songs.', ' It was launched in Hyderabad in March 2006, in C

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

In [8]:
doc = dataset['train'][0]['context']

In [9]:
print(doc)

{'title': ['Radio City (Indian radio station)', 'History of Albanian football', 'Echosmith', "Women's colleges in the Southern United States", 'First Arthur County Courthouse and Jail', "Arthur's Magazine", '2014–15 Ukrainian Hockey Championship', 'First for Women', 'Freeway Complex Fire', 'William Rast'], 'sentences': [["Radio City is India's first private FM radio station and was started on 3 July 2001.", ' It broadcasts on 91.1 (earlier 91.0 in most cities) megahertz from Mumbai (where it was started in 2004), Bengaluru (started first in 2001), Lucknow and New Delhi (since 2003).', ' It plays Hindi, English and regional songs.', ' It was launched in Hyderabad in March 2006, in Chennai on 7 July 2006 and in Visakhapatnam October 2007.', ' Radio City recently forayed into New Media in May 2008 with the launch of a music portal - PlanetRadiocity.com that offers music related news, videos, songs, and other music-related features.', ' The Radio station currently plays a mix of Hindi and 

In [10]:
docs = []
for title, sentences in zip(doc['title'], doc['sentences']):
    full_text = " ".join(sentences)
    split_docs = text_splitter.split_text(full_text)
    docs.extend(split_docs)

In [11]:
print("Number of chunks created: ", len(docs))
print("First chunk: ", docs[0])

Number of chunks created:  10
First chunk:  Radio City is India's first private FM radio station and was started on 3 July 2001.  It broadcasts on 91.1 (earlier 91.0 in most cities) megahertz from Mumbai (where it was started in 2004), Bengaluru (started first in 2001), Lucknow and New Delhi (since 2003).  It plays Hindi, English and regional songs.  It was launched in Hyderabad in March 2006, in Chennai on 7 July 2006 and in Visakhapatnam October 2007.  Radio City recently forayed into New Media in May 2008 with the launch of a music portal - PlanetRadiocity.com that offers music related news, videos, songs, and other music-related features.  The Radio station currently plays a mix of Hindi and Regional music.  Abraham Thomas is the CEO of the company.


In [12]:
!pip install langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 28.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.7 MB/s eta 0:00:00


In [13]:
from langchain.embeddings import SentenceTransformerEmbeddings
embeddings =SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")

<ipython-input-13-d04516a1f0c1>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings =SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [14]:
embeddingss = embeddings.embed_documents(docs)
embedding_array = np.array(embeddingss)
print("Embedding shape: ", embedding_array[0].shape)

Embedding shape:  (384,)


In [15]:
from langchain.vectorstores import FAISS
vectorstore = FAISS.from_texts(docs, embeddings)

In [16]:
vectorstore.add_texts(docs)

['a897b1eb-dbde-407a-a47d-4bc8bc8bbdc7',
 '2fe1d954-348c-4d12-a56a-ffda102a62a7',
 'cbd8c0a2-0ae9-4dee-8655-c416079dcdd7',
 '743e3386-1554-4785-b362-5c0ee584604a',
 'cae30577-47fe-4a12-a881-e2ef9a5409ce',
 '4251dea2-c92a-4834-b6ee-6e9f1f94eade',
 '49c2b80f-57f9-4639-a5d4-9ab5f205dc05',
 '1dd8ebca-4dc8-4703-9b3b-c92d29154f61',
 'c4fe731d-a51a-4020-9da0-509e8ad26f89',
 'be8884c3-cd30-4945-8978-42beff0d1137']

In [17]:
retriever = vectorstore.as_retriever()

In [18]:
query = "Which magazine was started first Arthur's Magazine or First for Women?"
retrieved_docs = retriever.get_relevant_documents(query, k=5)

<ipython-input-18-1524772fb05e>:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(query, k=5)


In [19]:
print("Top 5 retrieved passages for query: ", query)
for i, doc in enumerate(retrieved_docs):
    print(f"Passage {i+1}: {doc}")

Top 5 retrieved passages for query:  Which magazine was started first Arthur's Magazine or First for Women?
Passage 1: page_content='Arthur's Magazine (1844–1846) was an American literary periodical published in Philadelphia in the 19th century.  Edited by T.S. Arthur, it featured work by Edgar A. Poe, J.H. Ingraham, Sarah Josepha Hale, Thomas G. Spear, and others.  In May 1846 it was merged into "Godey's Lady's Book".'
Passage 2: page_content='Arthur's Magazine (1844–1846) was an American literary periodical published in Philadelphia in the 19th century.  Edited by T.S. Arthur, it featured work by Edgar A. Poe, J.H. Ingraham, Sarah Josepha Hale, Thomas G. Spear, and others.  In May 1846 it was merged into "Godey's Lady's Book".'
Passage 3: page_content='First for Women is a woman's magazine published by Bauer Media Group in the USA.  The magazine was started in 1989.  It is based in Englewood Cliffs, New Jersey.  In 2011 the circulation of the magazine was 1,310,696 copies.'
Passage 4

In [21]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

In [28]:
def rerank_passages(query, passages, reranker_model, reranker_tokenizer):
    # Prepare query and passage inputs
    # Assuming 'Document' objects have a 'page_content' attribute containing the passage text
    # If your Document objects have a different attribute for the text,
    # replace 'page_content' with the correct attribute name
    inputs = [query + " [SEP] " + passage.page_content for passage in passages]
    tokenized_inputs = reranker_tokenizer(inputs, padding=True, truncation=True, return_tensors="pt")

    # Perform inference to get relevance scores
    with torch.no_grad():
        outputs = reranker_model(**tokenized_inputs)
        scores = outputs.logits.squeeze(-1).tolist()  # Get the logits for each passage

    # Sort passages based on the scores in descending order
    # Use the original 'passage' object in the ranking, not just the text
    ranked_passages = sorted(zip(passages, scores), key=lambda x: x[1], reverse=True)

    # Return the ranked passages along with their scores
    return [(passage, score) for passage, score in ranked_passages]

In [26]:
reranker_model, reranker_tokenizer = load_reranker('cross-encoder/ms-marco-MiniLM-L-12-v2')



/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [29]:
reranked_passages = rerank_passages(query, retrieved_docs, reranker_model, reranker_tokenizer)

In [30]:
print("Reranked passages based on relevance scores:")
for i, (passage, score) in enumerate(reranked_passages):
    print(f"Rank {i+1}: {passage} (Score: {score})")

Reranked passages based on relevance scores:
Rank 1: page_content='Arthur's Magazine (1844–1846) was an American literary periodical published in Philadelphia in the 19th century.  Edited by T.S. Arthur, it featured work by Edgar A. Poe, J.H. Ingraham, Sarah Josepha Hale, Thomas G. Spear, and others.  In May 1846 it was merged into "Godey's Lady's Book".' (Score: 4.434252738952637)
Rank 2: page_content='Arthur's Magazine (1844–1846) was an American literary periodical published in Philadelphia in the 19th century.  Edited by T.S. Arthur, it featured work by Edgar A. Poe, J.H. Ingraham, Sarah Josepha Hale, Thomas G. Spear, and others.  In May 1846 it was merged into "Godey's Lady's Book".' (Score: 4.434252738952637)
Rank 3: page_content='First for Women is a woman's magazine published by Bauer Media Group in the USA.  The magazine was started in 1989.  It is based in Englewood Cliffs, New Jersey.  In 2011 the circulation of the magazine was 1,310,696 copies.' (Score: 4.126127243041992)


In [36]:
!pip install rank_bm25 datasets


In [34]:
!pip install rank_bm25
!pip install datasets

In [37]:
from datasets import load_metric
from rank_bm25 import BM25Okapi
import numpy as np

ImportError: cannot import name 'load_metric' from 'datasets' (/usr/local/lib/python3.10/dist-packages/datasets/__init__.py)